# Whole-genome analysis workflow

In [2]:
# ~2 minutes to install 
#%pip install -U --no-cache-dir scikit-learn scikit-optimize prefect prefect-ray ray plotly openpyxl shap lion_pytorch pytorch_tabnet xgboost neptune pyspark pyarrow dill fastnumbers

In [24]:
from prefect import task, flow
from prefect.task_runners import ConcurrentTaskRunner
from prefect_ray.task_runners import RayTaskRunner
import ray

import pandas as pd
import numpy as np

import logging

!export PREFECT_LOGGING_LEVEL="WARNING"
ray.shutdown()
parallelRunner = ray.init(
  configure_logging=True,
  logging_level=logging.ERROR,
)
parallelRunner

67841.43s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Python version:,3.10.10
Ray version:,2.5.1


In [25]:
from sklearn.ensemble import (
    AdaBoostClassifier,
    RandomForestClassifier,
)
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC, SVC
from xgboost import XGBClassifier

from skopt.space import Categorical, Integer, Real

from env import neptune_api_token

RadialBasisSVC = SVC
RadialBasisSVC.__name__ = "RadialBasisSVC"

clearHistory = False


config = {
    "vcfLike": {
        "path": "../notebook/Variant_report_NUPs_fixed_2022-03-28.xlsx",  # variant call table with annotations
        "sheet": "all cases vs all controls",  # sheet name if Excel spreadsheet
        "indexColumn": [
            "chrom",
            "position",
            "Gene",
        ],  # header that indexes variants (set as list with multiple columns)
        "compoundSampleIdDelimiter": "__",  # delimiter for compound sample IDs in column names
        "compoundSampleIdStartIndex": 1,  # index of first sample ID in compound sample ID
        "binarize": True,  # binarize variants to 0/1, or sum to weigh allele frequency
        "minAlleleFrequency": 0.05,  # filter out variants with allele frequency less than this
        # 'alleleModel': ['dominant', 'recessive', 'overDominant'],  # biallelic allele models to test on gene sets
        "filters": {},
    },  # TODO handle genotypes from related individuals
    "geneSets": {},  # TODO gene sets
    "tracking": {
        "name": "Nucleoporin genes",  # name of the experiment
        "entity": "ejmockler",
        "project": "ALS-NUPS-60__1",
        "plotAllSampleImportances": True,  # if calculating Shapely explanations, plot each sample in Neptune
        "token": neptune_api_token,
        "remote": False,  # if True, log to Neptune
    },
    "clinicalTable": {
        "path": "../notebook/ACWM.xlsx",  # clinical data as Excel spreadsheet
        "idColumn": "ExternalSampleId",  # genotype ID header
        "subjectIdColumn": "ExternalSubjectId",  # unique ID for each patient
        "labelColumn": "Subject Group",  # header that has case/control labels
        "controlLabels": [
            "Non-Neurological Control"
        ],  # these labels include external sample IDs (like 1000 Genomes)
        "caseLabels": ["ALS Spectrum MND"],  # "ALS Spectrum MND"
        "controlAlias": "control",
        "caseAlias": "case",
        "filters": "pct_european>=0.85",  # filter out nonhomogenous samples with less than 85% European ancestry
    },
    "externalTables": {
        "path": [
            "../notebook/igsr-1000 genomes phase 3 release.tsv",
            # "../notebook/ALS-NUPS-2000__accurateSamples_>=97.5%.csv",
            "../notebook/ACWM_ethnicallyVariable.tsv",
            "../notebook/ACWM_ethnicallyVariable.tsv",
            "../notebook/igsr-1000 genomes phase 3 release.tsv",
        ],  # external sample table
        "label": [
            "control",
            # "case",
            "case",
            "control",
            "control",
        ],  # case | control
        "setType": [
            "crossval",
            # "crossval",
            "holdout",
            "holdout",
            "holdout",
        ],
        "idColumn": [
            "Sample name",
            # "id",
            "ExternalSubjectId",
            "ExternalSubjectId",
            "Sample name",
        ],  # sample ID header
        "filters": [
            "`Superpopulation code`=='EUR' & `Population name`!='Finnish'",  # remove finnish samples due to unusual homogeneity (verify w/ PCA)
            # "`testLabel`==1",
            "`Subject Group`=='ALS Spectrum MND' & `pct_european`<0.85",
            "`Subject Group`=='Non-Neurological Control' & `pct_european`<0.85",
            "`Superpopulation code`!='EUR' & `Population name`!='Finnish'",
        ],
    },
    "sampling": {
        "bootstrapIterations": 50,
        "crossValIterations": 10,  # number of validations per bootstrap iteration
        "holdoutSplit": 0.1,
        "lastIteration": 50,
        "sequesteredIDs": [],
    },
    "model": {
        "hyperparameterOptimization": True,
        "calculateShapelyExplanations": False,
    },
}

 
async def remove_all_flows():
  from prefect.client import get_client
  orion_client = get_client()
  flows = await orion_client.read_flows()
  for flow in flows:
    flow_id = flow.id
    print(f"Deleting flow: {flow.name}, {flow_id}")
    await orion_client._client.delete(f"/flows/{flow_id}")
    print(f"Flow with UUID {flow_id} deleted")

if clearHistory: await remove_all_flows()

In [ ]:
from prefect import unmapped
from tqdm import tqdm

from tasks.input import processInputFiles

(caseGenotypes,
caseIDs,
holdoutCaseGenotypes,
holdoutCaseIDs,
controlGenotypes,
controlIDs,
holdoutControlGenotypes,
holdoutControlIDs,
clinicalData) = processInputFiles(config)

print(f"\nclinical data:\n{clinicalData.head()}")

10:14:20.032 | INFO    | prefect.engine - Created flow run 'obedient-sheep' for flow 'processInputFiles'

10:14:20.035 | INFO    | Flow run 'obedient-sheep' - View at http://127.0.0.1:4200/flow-runs/flow-run/e59c91bd-a3cd-46d4-b384-edd017fccc69

10:14:37.720 | INFO    | Flow run 'obedient-sheep' - Created task run 'load-0' for task 'load'

10:14:37.754 | INFO    | Flow run 'obedient-sheep' - Executing 'load-0' immediately...

10:18:44.096 | ERROR   | Task run 'load-0' - Crash detected! Execution was cancelled by the runtime environment.

...

10:20:00.700 | INFO    | Flow run 'obedient-sheep' -   File "/home/noot/.pyenv/versions/miniconda3-latest/lib/python3.10/site-packages/httpx/_client.py", line 1719, in _send_single_request
    response = await transport.handle_async_request(request)

10:20:00.701 | INFO    | Flow run 'obedient-sheep' -   File "/home/noot/.pyenv/versions/miniconda3-latest/lib/python3.10/site-packages/httpx/_transports/default.py", line 352, in handle_async_request
    with map_httpcore_exceptions():

10:20:00.702 | INFO    | Flow run 'obedient-sheep' -   File "/home/noot/.pyenv/versions/miniconda3-latest/lib/python3.10/contextlib.py", line 153, in __exit__
    self.gen.throw(typ, value, traceback)

10:20:00.704 | INFO    | Flow run 'obedient-sheep' -   File "/home/noot/.pyenv/versions/miniconda3-latest/lib/python3.10/site-packages/httpx/_transports/default.py", line 77, in map_httpcore_exceptions
    raise mapped_exc(message) from exc

10:20:00.705 | INFO    | Flow run 'obedient-sheep' - httpx.ConnectError: All connection attempts failed

In [17]:
def findBaselineFeature(caseGenotypes, controlGenotypes):
    # calculate the mean of each feature for cases and controls
    mean_cases = caseGenotypes.mean(axis=1)
    mean_controls = controlGenotypes.mean(axis=1)

    # calculate the absolute difference in means for each feature
    diff_means = abs(mean_cases - mean_controls)

    # get the feature with the largest difference in means
    selected_feature = diff_means.idxmax()

    print("Selected Feature for baseline perplexity: ", selected_feature)
    return selected_feature

caseGenotypes.loc[findBaselineFeature(caseGenotypes, controlGenotypes)]


Selected Feature for baseline perplexity:  ('6', '17675015', 'NUP153')


ALS__CGND-HDA-04091__NEUHF998PCY         1.0
aals-ALS__CGND-HDA-04089__NEUEU419NMF    1.0
aals-ALS__CGND-HDA-04086__NEUDH813DE6    1.0
aals-ALS__CGND-HDA-04085__NEUXZ486GG5    0.0
aals-ALS__CGND-HDA-04084__NEUHZ364FZW    0.0
                                        ... 
ALS__CGND-HDA-00013__UP-WGS-196          1.0
ALS__CGND-HDA-00012__UP-WGS-195          1.0
ALS__CGND-HDA-00008__UP-WGS-191          0.0
ALS__CGND-HDA-00004__UP-WGS-187          1.0
ALS__CGND-HDA-00001__UP-WGS-185          0.0
Name: (6, 17675015, NUP153), Length: 2052, dtype: float64

In [17]:
from joblib import Parallel, delayed
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

from models import stack as modelStack
from mlStack import bootstrap, serializeBootstrapResults

from metaconfig import metaconfig


def relativePerplexity(y_true, y_pred, y_true_baseline, y_pred_baseline, epsilon=1e-15):
    samplePerplexity = perplexity(y_true, y_pred)
    baselineSamplePerplexity = perplexity(y_true_baseline, y_pred_baseline)
    
    return pd.Series([samplePerplexity, baselineSamplePerplexity, np.divide(
            samplePerplexity, baselineSamplePerplexity + epsilon
        )])  # relative perplexity = perplexity / perplexity of model with single-most case correlated feature

def getBaselineFeatureResults(
    caseGenotypes,
    controlGenotypes,
    holdoutCaseGenotypes,
    holdoutControlGenotypes,
    clinicalData,
    config
):
    selectedFeature = findBaselineFeature(caseGenotypes, controlGenotypes)
    outerCvIterator = StratifiedKFold(
        n_splits=config["sampling"]["crossValIterations"], shuffle=False
    )
    innerCvIterator = outerCvIterator

    bootstrap_args = [
        (
            caseGenotypes.loc[[selectedFeature]],
            controlGenotypes.loc[[selectedFeature]],
            holdoutCaseGenotypes.loc[[selectedFeature]],
            holdoutControlGenotypes.loc[[selectedFeature]],
            clinicalData,
            model,
            hyperParameterSpace,
            innerCvIterator,
            outerCvIterator,
            config,
            False,  # disable tracking
        )
        for model, hyperParameterSpace in list(modelStack.items())
    ]
    results = Parallel(n_jobs=-1)(delayed(bootstrap)(*args) for args in bootstrap_args)

    baselineFeatureResults = {}
    for i in range(len(modelStack)):
        modelResults = results[i]
        baselineFeatureResults = serializeBootstrapResults(
            modelResults, baselineFeatureResults
        )
    baselineFeatureResultsDataframe = pd.DataFrame.from_dict(
        baselineFeatureResults,
        orient="index",
        columns=["label", "probability", "accuracy"],
    )
    baselineFeatureResultsDataframe.index.name = "id"

    return baselineFeatureResultsDataframe, selectedFeature


def perplexity(y_true, y_pred):
    crossEntropy = log_loss(
        y_true, y_pred, labels=[0, 1], eps=1e-15
    )  # linear predictions (exactly 0 or 1) depend on offset of 1e-15 when log is applied to avoid NaN

    # perplexity = 2 ^ crossEntropy
    return np.power(2, crossEntropy)


def findBaselineFeature(caseGenotypes, controlGenotypes):
    # calculate the mean of each feature for cases and controls
    mean_cases = caseGenotypes.mean(axis=1)
    mean_controls = controlGenotypes.mean(axis=1)

    # calculate the absolute difference in means for each feature
    diff_means = abs(mean_cases - mean_controls)

    # get the feature with the largest difference in means
    selected_feature = diff_means.idxmax()

    print("Selected feature for baseline perplexity: ", selected_feature)
    return selected_feature


currentResults = pd.read_csv(
            f"projects/{config['tracking']['project']}__1/sampleResults.csv",
            index_col="id",
        )
baselineFeatureResults, selectedFeature = getBaselineFeatureResults(
    caseGenotypes,
    controlGenotypes,
    holdoutCaseGenotypes,
    holdoutControlGenotypes,
    clinicalData,
    config
)
# serialize probability arrays from string
currentResults["probability"] = currentResults["probability"].apply(lambda x: np.array(eval(x))[:, 1])
# take intersection of bootstrapped samples
currentResults = currentResults.loc[baselineFeatureResults.index.intersection(currentResults.index)]
baselineFeatureResults = baselineFeatureResults.loc[currentResults.index]
currentResults["baselineProbability"] = baselineFeatureResults["probability"]

relativePerplexities = pd.DataFrame(index=currentResults.index)
new_cols = currentResults.apply(
    lambda row: relativePerplexity(
        [row["label"]] * len(row["probability"]),
        row["probability"],
        [row["label"]] * len(row["baselineProbability"]),
        row["baselineProbability"],
    ),
    axis=1,
    result_type='expand'
)

relativePerplexities["all features"], relativePerplexities[f"{selectedFeature}"], relativePerplexities["relative"] = new_cols[0], new_cols[1], new_cols[2]


FileNotFoundError: [Errno 2] No such file or directory: 'projects/ALS-NUPS-60__1/sampleResults.csv'

In [47]:
relativePerplexities

,all features,"('6', '17675015', 'NUP153')",relative
id,,,
ALS__CGND-HDA-04091__NEUHF998PCY,1.324706,4027.917073,0.000329
aals-ALS__CGND-HDA-04089__NEUEU419NMF,1.146718,159.976571,0.007168
aals-ALS__CGND-HDA-04083__NEURN392PGA,4569.180185,42.984855,106.297443
aals-ALS__CGND-HDA-04079__NEUMT573TE9,212899.177204,569.180350,374.045199
aals-ALS__CGND-HDA-04073__NEUBZ512CWM,1.593855,571.154062,0.002791
...,...,...,...
NA19059,1.247831,1.614929,0.772685
NA19080,161.349464,1.614929,99.911170
NA19143,1.637351,1.614929,1.013884


In [26]:
currentResults.index

Index(['ALS__CGND-HDA-04091__NEUHF998PCY',
       'aals-ALS__CGND-HDA-04086__NEUDH813DE6',
       'aals-ALS__CGND-HDA-04083__NEURN392PGA',
       'aals-ALS__CGND-HDA-04082__NEUTB997GDW',
       'aals-ALS__CGND-HDA-04081__NEUAD952KAZ',
       'aals-ALS__CGND-HDA-04079__NEUMT573TE9',
       'aals-ALS__CGND-HDA-04078__NEUTE443RWG',
       'aals-ALS__CGND-HDA-04069__NEUEA668FYK',
       'aals-ALS__CGND-HDA-04068__NEUAX021NPV',
       'aals-ALS__CGND-HDA-04067__NEUJA207UUV',
       ...
       'NA20877', 'NA20906', 'NA19376', 'NA18950', 'NA18998', 'NA19908',
       'NA20126', 'NA20859', 'ALS__CGND-HDA-00651__MH-WASHU-29',
       'ALS__CGND-HDA-00314__162ALS'],
      dtype='object', name='id', length=4606)

In [27]:
baselineFeatureResults.index

Index(['aals-ALS__CGND-HDA-04082__NEUTB997GDW',
       'aals-ALS__CGND-HDA-04081__NEUAD952KAZ',
       'aals-ALS__CGND-HDA-04079__NEUMT573TE9',
       'aals-ALS__CGND-HDA-04075__NEUZA643DHA',
       'aals-ALS__CGND-HDA-04069__NEUEA668FYK',
       'aals-ALS__CGND-HDA-04064__NEUKW840TXJ',
       'ALS__CGND-HDA-04049__NEURN540KF7', 'ALS__CGND-HDA-04048__NEUVR250XF0',
       'ALS__CGND-HDA-04045__NEUWE409BEK', 'ALS__CGND-HDA-04044__NEUEM180ZTU',
       ...
       'HG03789', 'NA18614', 'HG04225', 'NA19446', 'NA19129', 'NA19131',
       'NA19782', 'NA19908', 'ALS__CGND-HDA-00925__MH-WASHU-303',
       'ALS__CGND-HDA-00795__MH-WASHU-173'],
      dtype='object', name='id', length=4615)

In [4]:
len(holdoutControlIDs)

2012

## Evaluate model stack

In [ ]:
import neptune
from sklearn.model_selection import StratifiedKFold
from mlStack import classify
from config import config

outerCvIterator = StratifiedKFold(
    n_splits=config["sampling"]["crossValIterations"], shuffle=False
)
innerCvIterator = outerCvIterator
if config["tracking"]["remote"]:
    projectTracker = neptune.init_project(
        project=f'{config["tracking"]["entity"]}/{config["tracking"]["project"]}',
        api_token=config["tracking"]["token"],
    )

results = []
for model, hyperParameterSpace in list(config["model"]["stack"].items()):
    results.append(
        await classify(
            caseGenotypes,
            controlGenotypes,
            holdoutCaseGenotypes,
            holdoutControlGenotypes,
            clinicalData,
            model,
            hyperParameterSpace,
            innerCvIterator,
            outerCvIterator,
        ),
    )

In [23]:
from mlStack import main as runMLStack

results = runMLStack(
        config, 
        caseGenotypes,
        caseIDs,
        holdoutCaseGenotypes,
        holdoutCaseIDs,
        controlGenotypes,
        controlIDs,
        holdoutControlGenotypes,
        holdoutControlIDs,
        clinicalData,)

09:38:05.970 | INFO    | prefect.engine - Created flow run 'fractal-pillbug' for flow 'main'

09:38:05.971 | INFO    | Flow run 'fractal-pillbug' - View at http://127.0.0.1:4200/flow-runs/flow-run/9608ea9c-661b-49b4-a811-c5c7c42fcb06

09:38:05.973 | INFO    | prefect.task_runner.ray - Local Ray instance is already initialized. Using existing local instance.

65976.97s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
65977.14s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
65977.33s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
65977.51s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
65977.70s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
65977.88s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
65978.06s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
65978.25s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
65978.42s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
65978.60s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
65978.78s - pydevd: Sending message rela

09:48:39.678 | ERROR   | Flow run 'fractal-pillbug' - Encountered exception during execution:
Traceback (most recent call last):
  File "/home/noot/.pyenv/versions/miniconda3-latest/lib/python3.10/site-packages/prefect/engine.py", line 833, in orchestrate_flow_run
    result = await flow_call.aresult()
  File "/home/noot/.pyenv/versions/miniconda3-latest/lib/python3.10/site-packages/prefect/_internal/concurrency/calls.py", line 292, in aresult
    return await asyncio.wrap_future(self.future)
  File "/home/noot/.pyenv/versions/miniconda3-latest/lib/python3.10/site-packages/prefect/_internal/concurrency/calls.py", line 316, in _run_sync
    result = self.fn(*self.args, **self.kwargs)
  File "/home/noot/Desktop/machine-learning-als-genomics/src/mlStack.py", line 1243, in main
    accuracyHistogram = px.histogram(
  File "/home/noot/.pyenv/versions/miniconda3-latest/lib/python3.10/site-packages/plotly/express/_chart_types.py", line 480, in histogram
    return make_figure(
  File "/home/noot/.pyenv/versions/miniconda3-latest/lib/python3.10/site-packages/plotly/express/_core.py", line 2009, in make_figure
    groups, orders = get_groups_and_orders(args, grouper)
  File "/home/noot/.pyenv/versions/miniconda3-latest/lib/python3.10/site-packages/plotly/express/_core.py", line 1950, in get_groups_and_orders
    unique_cache[col] = list(args["data_frame"][col].unique())
  File "/home/noot/.pyenv/versions/miniconda3-latest/lib/python3.10/site-packages/pandas/core/series.py", line 2242, in unique
    return super().unique()
  File "/home/noot/.pyenv/versions/miniconda3-latest/lib/python3.10/site-packages/pandas/core/base.py", line 1001, in unique
    result = unique1d(values)
  File "/home/noot/.pyenv/versions/miniconda3-latest/lib/python3.10/site-packages/pandas/core/algorithms.py", line 409, in unique
    return unique_with_mask(values)
  File "/home/noot/.pyenv/versions/miniconda3-latest/lib/python3.10/site-packages/pandas/core/algorithms.py", line 425, in unique_with_mask
    uniques = table.unique(values)
  File "pandas/_libs/hashtable_class_helper.pxi", line 5910, in pandas._libs.hashtable.PyObjectHashTable.unique
  File "pandas/_libs/hashtable_class_helper.pxi", line 5857, in pandas._libs.hashtable.PyObjectHashTable._unique
TypeError: unhashable type: 'numpy.ndarray'

09:48:39.882 | ERROR   | Flow run 'fractal-pillbug' - Finished in state Failed("Flow run encountered an exception. TypeError: unhashable type: 'numpy.ndarray'")

TypeError: unhashable type: 'numpy.ndarray'

In [6]:
from sklearn import datasets
X, y  = datasets.make_classification(n_samples=400)

def train_data(model, X=X, y=y):
    clf = model
    clf.fit(X, y)
    
    
from sklearn.linear_model import LogisticRegression 
from cuml.linear_model import LogisticRegression as LogisticRegression_gpu

sklearn_time_svc = %timeit -o train_data(LogisticRegression(penalty="l2", solver="saga"))

from cuml.common.device_selection import using_device_type
with using_device_type('gpu'):
    cuml_time_svc = %timeit -o train_data(LogisticRegression_gpu(penalty="l2"))

print(f"""Average time of sklearn's {LogisticRegression.__name__}""", sklearn_time_svc.average, 's')
print(f"""Average time of cuml's {LogisticRegression_gpu.__name__}""", cuml_time_svc.average, 's')

print('Ratio between sklearn and cuml is', sklearn_time_svc.average/cuml_time_svc.average)

2.29 ms ± 131 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
7.53 ms ± 1.16 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Average time of sklearn's LogisticRegression 0.002288513623003382 s
Average time of cuml's LogisticRegression 0.007530984568542668 s
Ratio between sklearn and cuml is 0.30387973872136553
